In order to run this notebook, the following environment and packages had to be installed:

$ conda create -n v-env python=3.8

activate v-env

conda activate v-env

conda config --set channel_priority false

conda install -c conda-forge prophet

pip install salesforce-merlion

pip install pingouin

pip install PyYAML

pip install SQLAlchemy

pip install ipykernel

pip install jinja2

pip install sfc-merlion[plot]

pip install -e ts_datasets/

conda install -e ts_datasets/

conda install -c conda-forge lightgbm

conda activate v-env

pip install -e ts_datasets/

pip intall -e ./Merlion/ts_dataset/

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import norm

#from ts_datasets.anomaly import NAB
#from ts_datasets.forecast import M4

#from merlion.plot import plot_anoms
from merlion.utils import TimeSeries
from merlion.utils.time_series import TimeSeries

from merlion.transform.base import Identity
from merlion.transform.resample import TemporalResample
from merlion.transform.moving_average import DifferenceTransform

from merlion.models.defaults import DefaultDetectorConfig, DefaultDetector
#from merlion.models.anomaly.isolation_forest import IsolationForest, IsolationForestConfig
#from merlion.models.anomaly.windstats import WindStats, WindStatsConfig
#from merlion.models.anomaly.forecast_based.prophet import ProphetDetector, ProphetDetectorConfig

from merlion.models.forecast.arima import Arima, ArimaConfig
from merlion.models.forecast.prophet import Prophet, ProphetConfig
from merlion.models.forecast.smoother import MSES, MSESConfig

#from merlion.models.ensemble.anomaly import DetectorEnsemble, DetectorEnsembleConfig
from merlion.models.defaults import DefaultForecasterConfig, DefaultForecaster
from merlion.models.ensemble.combine import Mean, ModelSelector
from merlion.models.ensemble.forecast import ForecasterEnsemble, ForecasterEnsembleConfig

#from merlion.evaluate.anomaly import TSADMetric
from merlion.post_process.threshold import AggregateAlarms
from merlion.evaluate.forecast import ForecastMetric

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-3hiftyln because the default path (/home/mpingol/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
from merlion.evaluate.forecast import ForecastMetric
from merlion.models.factory import ModelFactory
from merlion.models.ensemble.combine import ModelSelector

# 1 Day Close and Tone 

In [3]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/netflix_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

In [4]:
tesla_mv = tesla[['Close', 'Daily Average Tone']]

In [5]:
tesla_mv

,Close,Daily Average Tone
Date,,
2015-02-19,67.800003,-0.452830
2015-02-20,68.314285,0.880952
2015-02-21,67.405716,0.530303
2015-02-22,67.839996,0.514706
2015-02-23,68.332855,-0.032258
...,...,...
2017-12-26,363.920013,0.427419
2017-12-27,367.649994,0.493617
2017-12-28,351.140015,0.864706


In [6]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

836 points in train split, 210 points in test split.
Time series is 2-dimensional


In [7]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [01:00<00:00, 298778.46it/s]


RMSE:  23.5043
sMAPE: 5.3738
MAE:   18.1910

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:07<00:00, 2458443.35it/s]


RMSE:  13.1725
sMAPE: 3.0415
MAE:   10.0673



# 1 Day Close Only

In [8]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/netflix_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

In [9]:
tesla_mv = tesla[['Close']]

In [10]:
tesla_mv

,Close
Date,
2015-02-19,67.800003
2015-02-20,68.314285
2015-02-21,67.405716
2015-02-22,67.839996
2015-02-23,68.332855
...,...
2017-12-26,363.920013
2017-12-27,367.649994
2017-12-28,351.140015


In [11]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

836 points in train split, 210 points in test split.
Time series is 1-dimensional


In [12]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:10<00:00, 1722517.63it/s]


RMSE:  17.8265
sMAPE: 4.2470
MAE:   14.0074

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2807944.11it/s]


RMSE:  13.1725
sMAPE: 3.0415
MAE:   10.0673



# 3 day close and tone

In [13]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/netflix_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close', 'Daily Average Tone']]

tesla_mv

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 70
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="3d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 2-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [01:03<00:00, 284583.30it/s]


RMSE:  18.6815
sMAPE: 5.2067
MAE:   17.6259

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:08<00:00, 2252309.21it/s]


RMSE:  11.3562
sMAPE: 3.0694
MAE:   10.1824



# 3 day close only

In [14]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/netflix_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close']]

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 1-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:10<00:00, 1771391.62it/s]


RMSE:  17.8265
sMAPE: 4.2470
MAE:   14.0074

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2755326.06it/s]


RMSE:  13.1725
sMAPE: 3.0415
MAE:   10.0673



# 5 day close and tone

In [15]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/netflix_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close', 'Daily Average Tone']]

tesla_mv

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 2-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [01:22<00:00, 218007.39it/s]


RMSE:  20.5215
sMAPE: 5.6475
MAE:   18.9982

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:07<00:00, 2326532.68it/s]


RMSE:  14.2670
sMAPE: 3.7825
MAE:   12.5240



# 5 day close only

In [16]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/netflix_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close']]

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 1-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:10<00:00, 1767132.79it/s]


RMSE:  18.6823
sMAPE: 5.1139
MAE:   16.8672

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2694690.94it/s]


RMSE:  14.2670
sMAPE: 3.7825
MAE:   12.5240

